# Desafio 

Criação de um crawler em python que coleta dados do instagram (post's e seus respectivos comentarios), dada uma hashtag como parâmentro, e armazena os dados em um banco de dados (preferencialmente mongoDB), o programa deverá ter a opção de pesquisa de dados no banco de dados de uma hashtag especifica.


> Dica:
Os dados do instagram podem ser retornados em formato json, para isso basta colocar o sufixo ```?_a=1``` no final da URL. 


Exeplo:
URL deposts contendo cruzeiro:

https://wwww.instagram.com/explore/tags/cruzeiro



mesma URL retornando os dados em json:


https://wwww.instagram.com/explore/tags/cruzeiro?_a=1



Para melhor vizualização dos dados json, há extensões de navegadores que melhoram a visuazação dos dados:
exemplo:


Json Formatter



Links úteis:
pythonforbeginners.com/requests/using-requests-in-python
mongodb.com/blog/post/getting-started-with-python-and-mongodb



In [10]:
#!pip install pymongo
import requests
import pandas as pd
from pymongo import MongoClient
import json

In [11]:
client = MongoClient('localhost',27017)
db = client.socialnetwork


In [12]:
def return_comments(shortcode_id):
    knowed_page = str('https://www.instagram.com/p/'+ shortcode_id+'/?__a=1')
    coments = json.loads(str(requests.get(knowed_page).text))
    total_comments = []
    for j in range(len(coments['graphql']['shortcode_media']['edge_media_to_parent_comment']['edges'])):
                   total_comments.append(coments['graphql']['shortcode_media']['edge_media_to_parent_comment']['edges'][j]['node']['text'])
    return total_comments



def crawling_page(newDictionary,searchedTerm):
    for i in range(len(newDictionary['graphql']['hashtag']['edge_hashtag_to_media']['edges'])):
        user_id = newDictionary['graphql']['hashtag']['edge_hashtag_to_media']['edges'][i]['node']['id']
        short_code = newDictionary['graphql']['hashtag']['edge_hashtag_to_media']['edges'][i]['node']['shortcode']
        text_post = newDictionary['graphql']['hashtag']['edge_hashtag_to_media']['edges'][i]['node']['edge_media_to_caption']['edges'][0]['node']['text']
        comments = list(return_comments(str(short_code)))
        
        userdata = {
            '_id':user_id,
            'short_code':short_code,
            'text_content':text_post,
            'comments':comments}
        
        db[str(searchedTerm)].insert_one(userdata)
           

In [13]:
def create_colection(tag):
    '''make a colection with the tag specified'''
    colection = db[str(tag)]
    return True

def register_new_tag():
    print('item a ser pesquisado:\n')
    searchedTerm = str(input())
    
    phraselSearchTerm = str('https://www.instagram.com/explore/tags/' + searchedTerm +'/?__a=1')
    
    pageTotal = requests.get(phraselSearchTerm)

    newDictionary=json.loads(str(pageTotal.text))
    
    create_colection(str(searchedTerm))
    
    crawling_page(newDictionary,searchedTerm)
    return None


def fetch(colection):
    for colect in colection.find():
        pprint.pprint(colect)        
    

def select_by_id(colection,id):
    return colection.find_one({'__id':id})


def delete_by_od(colection, id):
    return colection.delete_one({'_id':id})

def return_registered_colections():
     return db.list_collection_names()




    
                     
                     

In [14]:
register_new_tag()


item a ser pesquisado:

bbb20
